<a href="https://colab.research.google.com/github/janciesko/collab/blob/main/Kokkos_GPU_Programming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CUDA Hello World

In [2]:
%%writefile hello_cuda.cu
#include <iostream>
#include <cuda_runtime.h>

__global__ void helloFromGPU(void) {
  printf("Hello World from GPU!\n");
}

int main() {
  helloFromGPU<<<1, 1>>>();

  cudaError_t err = cudaGetLastError();
  if (err != cudaSuccess) {
    std::cerr << "CUDA error after kernel launch: " << cudaGetErrorString(err) << std::endl;
    return 1;
  }

  cudaDeviceSynchronize();  // Wait for GPU to finish

  err = cudaGetLastError();
  if (err != cudaSuccess) {
    std::cerr << "CUDA error after synchronization: " << cudaGetErrorString(err) << std::endl;
    return 1;
  }

  std::cout << "Hello World from CPU!\n";
  return 0;
}

Overwriting hello_cuda.cu


In [3]:
!nvcc -arch=sm_75 -o hello_cuda hello_cuda.cu
!./hello_cuda



Hello World from GPU!
Hello World from CPU!


In [4]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [5]:
!nvidia-smi


Wed Mar 19 18:04:19 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   36C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# Kokkos Hello World

In [7]:
!mkdir MyProject && cd MyProject

In [8]:
%%writefile MyProject/CMakeLists.txt

cmake_minimum_required(VERSION 3.16)
project(MyProject)

include(FetchContent)
FetchContent_Declare(
  Kokkos
  URL https://github.com/kokkos/kokkos/archive/refs/tags/4.5.01.zip
)
FetchContent_MakeAvailable(Kokkos)

add_executable(HelloKokkos HelloKokkos.cpp)
target_link_libraries(HelloKokkos Kokkos::kokkos)

Writing MyProject/CMakeLists.txt


In [9]:
%%writefile MyProject/HelloKokkos.cpp
#include <Kokkos_Core.hpp>

int main(int argc, char** argv) {
  Kokkos::initialize(argc, argv);
  {
    // Allocate a 1-dimensional view of integers
    Kokkos::View<int*> v("v", 5);
    // Fill view with sequentially increasing values v=[0,1,2,3,4]
    Kokkos::parallel_for("fill", 5, KOKKOS_LAMBDA(int i) { v(i) = i; });
    // Compute accumulated sum of v's elements r=0+1+2+3+4
    int r;
    Kokkos::parallel_reduce(
      "accumulate", 5,
      KOKKOS_LAMBDA(int i, int& partial_r) { partial_r += v(i); }, r);
    // Check the result
    KOKKOS_ASSERT(r == 10);
  }
  Kokkos::printf("Goodbye World\n");
  Kokkos::finalize();
  return 0;
}

Writing MyProject/HelloKokkos.cpp


In [10]:
!cd MyProject && cmake -B builddir -DKokkos_ENABLE_CUDA=ON

-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
CMake Warning (dev) at /usr/local/lib/python3.11/dist-packages/cmake/data/share/cmake-3.31/Modules/FetchContent.cmake:1373 (message):
  The DOWNLOAD_EXTRACT_TIMESTAMP option was not given and policy CMP0135 is
  not set.  The policy's OLD behavior will be used.  When using a URL
  download, the timestamps of extracted files should preferably be that of
  the time of extraction, otherwise code that depends on the extracted
  contents might not be rebuilt if the URL changes.  The OLD behavior

In [ ]:
!cd MyProject && cmake --build builddir

[  6%] Built target kokkossimd
[ 80%] Built target kokkoscore
[ 86%] Built target kokkoscontainers
[ 93%] Built target HelloKokkos
[ 93%] Built target AlwaysCheckGit
[100%] Built target impl_git_version


In [ ]:
!cd MyProject/builddir && ./HelloKokkos

Goodbye World
